In [6]:
import numpy as np
import tensorflow as tf
import pandas as pd

# Pre-processing functions
from src.data.data_helper import get_raw_data_as_dataframe
from src.models.preprocessing.preprocessor import SignalPreprocessor
from src.data.data_helper import segement_data

# Model functions
from src.models.LSTM.LSTM import LSTM
from src.models.LSTM_STFT.LSTM_STFT import LSTM_STFT
from src.models.LSTM_STFT_Dense.LSTM_STFT_Dense import LSTM_STFT_Dense

In [7]:
def get_training_data():
    raw_train, raw_val = get_raw_data_as_dataframe(
        validation_subjects=(1,2)
    )

    # Initialize the preprocessor
    pre_processor = SignalPreprocessor(
        low_freq=20.0,
        high_freq=500.0,
        fs=5000.0,
        order=7
    )
    # Calibrate the preprocessor
    pre_processor.calibrate(raw_train)

    window_length=200 * 5
    overlap=50 * 5

    seg_train = segement_data(
        raw_train, window_length=window_length, overlap=overlap
    )
    seg_val = segement_data(
        raw_val,   window_length=window_length, overlap=overlap
    )

    all_labels = pd.concat([seg_train['label'], seg_val['label']])
    num_classes = all_labels.nunique()

    y_train = tf.keras.utils.to_categorical(
        seg_train['label'].values, num_classes=num_classes
    )
    y_val = tf.keras.utils.to_categorical(
        seg_val['label'].values,  num_classes=num_classes
    )

    X_train = np.stack(seg_train.drop(columns=['label', 'source'])['window_data'].values)
    X_val   = np.stack(seg_val.drop(columns=['label', 'source'])['window_data'].values)

    X_train = pre_processor.batch_pre_process(X_train)
    X_val   = pre_processor.batch_pre_process(X_val)

    input_shape = X_train.shape[1]

    return X_train, y_train, X_val, y_val, num_classes, input_shape

In [8]:
X_train, y_train, X_val, y_val, num_classes, input_shape = get_training_data()

In [9]:
import keras_tuner as kt

In [10]:
class LSTMHyperModel(kt.HyperModel):

    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    # ----------------- build -----------------------------------------
    def build(self, hp):
        lr   = hp.Choice('learning_rate',
                         values=[1e-2, 2e-2, 5e-2, 1e-3, 2e-3, 5e-3, 1e-4, 2e-4, 5e-4])

        opt  = hp.Choice('optimizer',
                         values=['adam', 'rmsprop', 'nadam'])

        norm = hp.Choice('normalization',
                         values=['none', 'batch', 'layer'])

        # We only declare batch_size here; we'll use it in fit().
        hp.Choice('batch_size', values=[32, 64, 128, 256, 512])

        model = LSTM(self.input_shape,
                     self.num_classes,
                     learning_rate=lr,
                     optimizer=opt,
                     normalization=norm).get_model()
        return model

    # ----------------- fit -------------------------------------------
    def fit(self, hp, model, X_train, y_train, X_val, y_val, **kwargs):
        """
        Called by the tuner for every trial.  We inject the per-trial
        batch_size coming from hp.
        """
        batch_size = hp.get('batch_size')
        return model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            batch_size=batch_size,
            epochs=kwargs.get('epochs', 10),
            verbose=kwargs.get('verbose', 2)
        )

In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_score',      # metric name Keras assigns: ‘f1_score’
    mode='max',                  # we want to maximise it
    patience=5,
    restore_best_weights=True)

In [12]:
hypermodel = LSTMHyperModel(input_shape, num_classes)

In [13]:
from src.utils.path_utils import get_models_dir

model_dir = get_models_dir() / "LSTM_search"
model_dir

WindowsPath('C:/Users/erik/IES_codebase/EMG_Project/CDT406-Smart-Gripper/models/LSTM_search')

In [14]:
tuner = kt.BayesianOptimization(
    hypermodel,
    objective = kt.Objective("val_f1_score", direction="max"),
    max_trials=3,
    directory=model_dir,
    project_name="baseline_v2",
    overwrite=True
)

In [15]:
tuner.search(X_train, y_train,
             X_val=X_val, y_val=y_val,
             callbacks=[stop_early],
             epochs=30,
             verbose=2)

Trial 3 Complete [00h 00m 16s]
val_f1_score: 0.5690332651138306

Best val_f1_score So Far: 0.5690332651138306
Total elapsed time: 00h 00m 50s


In [16]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [17]:
# Retrieve the best trial’s hyper-parameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.

Optimal learning rate  : {best_hps.get('learning_rate')}
Optimal optimizer      : {best_hps.get('optimizer')}
Optimal normalization  : {best_hps.get('normalization')}
Optimal batch size     : {best_hps.get('batch_size')}
""")


The hyperparameter search is complete.

Optimal learning rate  : 0.0001
Optimal optimizer      : nadam
Optimal normalization  : batch
Optimal batch size     : 256

